In [108]:
#https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [109]:
print("b4 processing: " + str(len(df)))
#sklearn.preprocessing doesn't like nan
df = df.fillna(df['Age'].mode()[0])
#this numberic value screwes things up. There are 2 rows that have this value
df = df[df["Embarked"] != 24.0]
df = df.dropna()
print("aftr processing: " + str(len(df)))

b4 processing: 891
aftr processing: 889


In [110]:
# returns a DF that encodes the specified categorical variables.
from sklearn.preprocessing import LabelEncoder
def convertCat(aDFXVals,catCols):
    anotherDF = aDFXVals[catCols].apply(LabelEncoder().fit_transform)
    nonCatCols = aDFXVals.drop(catCols,axis=1)
    result = pd.concat([nonCatCols, anotherDF], axis=1)
    return result

In [134]:
features= [c for c in df.columns.values if c  not in ['PassengerId','Survived','Cabin','Name','Ticket']]

newDF = convertCat(df[features],['Sex','Embarked'])

In [135]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
my_pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

In [145]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
# the scoring function defaults to the one in the random forest classifier
scores = cross_val_score(my_pipeline, newDF, df["Survived"])
print(scores)
print('Mean Absolute Error %2f' %(1-scores.mean()))

[ 0.77777778  0.82094595  0.78040541]
Mean Absolute Error 0.206957


In [114]:
print("b4 processing: " + str(len(test)))
#sklearn.preprocessing doesn't like nan
test = test.fillna(test['Age'].mode()[0])
#this numberic value screwes things up. There are 2 rows that have this value
test = test[test["Embarked"] != 24.0]
test = test.dropna()
print("aftr processing: " + str(len(test)))

b4 processing: 418
aftr processing: 418


In [128]:
testDF = convertCat(test[features],['Sex','Embarked'])
my_pipeline.fit(newDF,df["Survived"])
predictions = my_pipeline.predict(testDF)

In [ ]:
my_submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
# you could use any filename. We choose submission here
my_submission.to_csv('submission_pipelines1.csv', index=False)

In [ ]:
#things to do:
#1. cross val